In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf


In [ ]:
# Mount Goggle drive to colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Imports
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.metrics import confusion_matrix
from tensorflow.keras import regularizers

from matplotlib import pyplot

In [ ]:
#load clean and normalized dataset
dataset = pd.read_csv(r'/content/drive/My Drive/echocardiogramKnnNorm.csv', index_col=0) 

In [ ]:
#Split dataset into train and test
train_knn=dataset.sample(frac=0.7,random_state=200) #random state is a seed value
test_knn=dataset.drop(train_knn.index)

In [ ]:
#shape of data
print(train_knn.shape)
print(test_knn.shape)

In [ ]:
#(Number of rows, Number of columns)
train_labels = train_knn.alive
train_data = train_knn.drop('alive',axis=1)
test_labels = test_knn.alive
test_data = test_knn.drop('alive',axis=1)

print(train_labels.shape)
print(test_labels.shape)
print(train_data.shape)
print(test_data.shape)

In [ ]:
# Load the TensorBoard notebook extension for visualization
%load_ext tensorboard

import datetime, os

In [ ]:
# Imports for early stopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Callbacks for early stopping and tensorboard
# checkpoint, save the best model
dataset_path = "/content/drive/My Drive/SP1 Proyect/"
filepath = dataset_path + "output.best.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')

# tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1,profile_batch=0)

#early stopping
stop_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=112)

# callbacks
callbacks_list = [checkpoint_callback,tensorboard_callback, stop_callback]

In [ ]:
# Run Tensorboad for monitoring
%tensorboard --logdir logs

In [ ]:
#build model, include L1 regularization in the model 
model = Sequential()

model.add(Dense(8, input_shape=(9,), activation='relu', activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(6, activation='relu', activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(4, activation='relu', activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(2, activation='relu', activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(1, activation='sigmoid')) # Sigmoid for binary clasification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
# Train Model
history = model.fit(train_data, 
          train_labels, 
          validation_split=0.2,
          epochs=4000,
          callbacks=callbacks_list)

In [ ]:
#plot training history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# evaluate the model, Final accuracy Train:88%m Test: 90%
_, train_acc = model.evaluate(train_data, train_labels, verbose=0)
_, test_acc = model.evaluate(test_data, test_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.882, Test: 0.900
